In [1]:
import pandas as pd
import itertools
from rater.models import Results, Experiment, People

In [2]:
def get_elo_scores(exp_id, num_runs = 500):
    
    def elo_match(ra, rb, winner = 'a', k = 32):
        exp_a = 1 / (1 + (10 ** ((rb - ra) / 400)))
        exp_b = 1 / (1 + (10 ** ((ra - rb) / 400)))
        if winner == 'a':
            fa = ra + (k * (1 - exp_a))
            fb = rb + (k * (0 - exp_b))
        else:
            fa = ra + (k * (0 - exp_a))
            fb = rb + (k * (1 - exp_b))

        return [fa, fb]
    
    my_exp = Experiment.objects.get(id = exp_id)
    my_res = Results.objects.filter(experiment_name = my_exp.id)
    df = pd.DataFrame.from_records(my_res.values('name_1', 'name_2','winner', 'rater'))
    df['loser'] = df.name_1
    df.loser[df.loser == df.winner] = df.name_2
    df.drop(['name_1', 'name_2'], axis = 1, inplace = True)
    
    all_runs = list()
    for i in range(num_runs):
        #shuffle data
        df = df.sample(frac = 1)
        #starting scores
        running_scores = {x: 1000 for x in set(pd.concat([df['winner'], df['loser']]))}
        #set scores based on wins/losses
        for index, row in df.iterrows():
            elo_update = elo_match(running_scores[row.winner], running_scores[row.loser])
            running_scores[row.winner] = elo_update[0]
            running_scores[row.loser] = elo_update[1]

        all_runs.append(pd.DataFrame(running_scores.items(), columns = ['name','score']))

    all_runs = pd.concat(all_runs, ignore_index = True)
    avg_scores = all_runs.groupby('name').mean()
    avg_scores.reset_index(drop = False, inplace = True)
    
    real_names = []
    for index, row in avg_scores.iterrows():
        real_names.append(str(People.objects.get(id = row['name'])))

    avg_scores['name'] = real_names
    
    return(avg_scores)

In [3]:
#exp = Experiment.objects.get(title = "TEST")
#exp.id

get_elo_scores(exp_id = 28)

,name,score
0,MAJ John Case,943.899246
1,COL Riley Post,999.821644
2,CPL Ian Kloo,1028.363382
3,PV2 TEST TEST,1027.915728


In [ ]:
def elo_match(ra, rb, winner = 'a', k = 32):
    exp_a = 1 / (1 + (10 ** ((rb - ra) / 400)))
    exp_b = 1 / (1 + (10 ** ((ra - rb) / 400)))
    if winner == 'a':
        fa = ra + (k * (1 - exp_a))
        fb = rb + (k * (0 - exp_b))
    else:
        fa = ra + (k * (0 - exp_a))
        fb = rb + (k * (1 - exp_b))
        
    return [fa, fb]

In [ ]:
#get experiment - this one is known complete
my_exp = Experiment.objects.get(title = "TEST")
my_exp

In [ ]:
#get results for that experiment
my_res = Results.objects.filter(experiment_name = my_exp.id)
my_res

In [ ]:
#make pandas dataframe out of query results and establish loser column
df = pd.DataFrame.from_records(my_res.values('name_1', 'name_2','winner', 'rater'))
df['loser'] = df.name_1
df.loser[df.loser == df.winner] = df.name_2
df.drop(['name_1', 'name_2'], axis = 1, inplace = True)

df

In [ ]:
#adjudicate elo ratings
num_runs = 500

all_runs = list()
for i in range(num_runs):
    #shuffle data
    df = df.sample(frac = 1)
    #starting scores
    running_scores = {x: 1000 for x in set(pd.concat([df['winner'], df['loser']]))}
    #set scores based on wins/losses
    for index, row in df.iterrows():
        elo_update = elo_match(running_scores[row.winner], running_scores[row.loser])
        running_scores[row.winner] = elo_update[0]
        running_scores[row.loser] = elo_update[1]

    all_runs.append(pd.DataFrame(running_scores.items(), columns = ['name','score']))

all_runs = pd.concat(all_runs, ignore_index = True)
avg_scores = all_runs.groupby('name').mean()
avg_scores.reset_index(drop = False, inplace = True)

avg_scores

In [ ]:
#pull names
real_names = []
for index, row in avg_scores.iterrows():
    real_names.append(str(People.objects.get(id = row['name'])))

avg_scores['name'] = real_names
avg_scores